In [ ]:
!pip install vanna[bigquery]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
import vanna
from vanna.remote import VannaDefault

In [ ]:
# put the email address you used to sign up at vanna.ai
api_key = vanna.get_api_key('xxxxxxxxx@gmail.com')

Check your email for the code and enter it here: 46QIGH


In [ ]:
# put the model name you created at vanna.ai for the specific dataset you want to analyze
vanna_model_name = 'jobposts_ex'

In [ ]:
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

In [ ]:
ddl_statements = """
CREATE TABLE `hultaibigdata.jobs_data_analyst.postings`
(
  title STRING,
  company_name STRING,
  location STRING,
  via STRING,
  description STRING,
  extensions STRING,
  posted_at STRING,
  schedule_type STRING,
  work_from_home BOOL,
  salary STRING,
  search_term STRING,
  search_location STRING,
  commute_time FLOAT64,
  salary_pay STRING,
  salary_rate STRING,
  salary_avg FLOAT64,
  salary_min FLOAT64,
  salary_max FLOAT64,
  salary_hourly FLOAT64,
  salary_yearly FLOAT64,
  salary_standardized FLOAT64,
  description_tokens STRUCT<list ARRAY<STRUCT<element STRING>>>,
  index INT64,
  date_time TIMESTAMP
);
"""

In [ ]:
# training based on the table schema
vn.train(ddl = ddl_statements)

Adding ddl: 
CREATE TABLE `hultaibigdata.jobs_data_analyst.postings`
(
  title STRING,
  company_name STRING,
  location STRING,
  via STRING,
  description STRING,
  extensions STRING,
  posted_at STRING,
  schedule_type STRING,
  work_from_home BOOL,
  salary STRING,
  search_term STRING,
  search_location STRING,
  commute_time FLOAT64,
  salary_pay STRING,
  salary_rate STRING,
  salary_avg FLOAT64,
  salary_min FLOAT64,
  salary_max FLOAT64,
  salary_hourly FLOAT64,
  salary_yearly FLOAT64,
  salary_standardized FLOAT64,
  description_tokens STRUCT<list ARRAY<STRUCT<element STRING>>>,
  index INT64,
  date_time TIMESTAMP
);



'178673-ddl'

In [12]:
vn.train(question = "What are the top 10 most commonly mentioned skills that have average price more than 80000 and title have word senior", sql="""
WITH current_data as(
  SELECT
  *
FROM`hultaibigdata.jobs_data_analyst.postings`
WHERE LOWER(title) LIKE '%senior%'
  AND salary_avg > 80000
)
SELECT
  list.element AS skill, COUNT(*) AS frequency
FROM
  current_data, UNNEST(description_tokens.list) AS list
GROUP BY skilla
ORDER BY frequency DESC
LIMIT 10;
""")

'520b0d2f60918a2d2b2f517dd6906d60-sql'

In [ ]:
vn.train(question = "What percent of job posts provide salary information?", sql="""
SELECT
  100*count(salary_standardized) / count(*)
FROM
  `hultaibigdata.jobs_data_analyst.postings`;
""")

'dd5d0e8144407382365dcfb0cc99345d-sql'

In [ ]:
vn.train(question = "What percent of job posts appearing since January 2023 provide salary information?", sql="""
SELECT
  100*count(salary_standardized) / count(*)
FROM
  `hultaibigdata.jobs_data_analyst.postings`
WHERE
  date_time > DATE('2023-01-01');
""")

'726162b9ac31754ecfce2fe4248b8ffa-sql'

In [ ]:
vn.train(question = "Show titles, company names, and dates of 5 most recent job posts", sql="""
SELECT
  title, company_name, date_time
FROM
  `hultaibigdata.jobs_data_analyst.postings`
ORDER BY date_time DESC
LIMIT 5;
""")

'35881cdb14b1869fe9e74fbbdbbd3edf-sql'

In [ ]:
vn.train(question = "Show the description and salary specification of the lowest paying job in the entire dataset", sql="""
SELECT
  description, salary
FROM
  `hultaibigdata.jobs_data_analyst.postings`
WHERE salary_standardized IS NOT NULL
ORDER BY salary_standardized ASC
LIMIT 1;
""")

'e3ad698ddb1d5f31e531addc4a19e1cd-sql'

In [ ]:
vn.train(question = "What is the mean annual salary among the posts where salary information is given?", sql="""
SELECT
  AVG(salary_standardized)
FROM
  `hultaibigdata.jobs_data_analyst.postings`;
""")

'80d6b1b12863730010c17ebfc103b21f-sql'

In [ ]:
vn.train(question = "What percent of job posts allow working from home (remote)?", sql="""
SELECT
  100*count(work_from_home) / count(*)
FROM
  `hultaibigdata.jobs_data_analyst.postings`;
""")

'c4516ef661dc85379664c2de8f695fc7-sql'

In [ ]:
vn.train(question = "What are the title, location, salary, and salary specification for the top 3 highest paying jobs in the entire dataset?", sql="""
SELECT
  title, location, salary_standardized, salary
FROM
  `hultaibigdata.jobs_data_analyst.postings`
ORDER BY salary_standardized DESC
LIMIT 3;
""")

'd3c6ff555e9b15913b87550739da95bd-sql'

In [ ]:
vn.train(question = "Show the top 5 listing channels for job posts with their post counts.", sql="""
SELECT
  via, count(*)
FROM
  `hultaibigdata.jobs_data_analyst.postings`
GROUP BY via
ORDER BY count(*) DESC
LIMIT 5;
""")

'96bdf09d227454cb96c5358294c7227b-sql'

In [ ]:
vn.train(question = "What types of schedule are represented in the dataset?", sql="""
SELECT
  DISTINCT schedule_type
FROM
  `hultaibigdata.jobs_data_analyst.postings`
""")

'5298101fe5e6fc6075c79792566b30f0-sql'

In [ ]:
vn.train(question = "What is the average salary of full-time jobs?", sql="""
SELECT
  AVG(salary_standardized)
FROM
  `hultaibigdata.jobs_data_analyst.postings`
WHERE
  schedule_type = 'Full-time';
""")

'747eced43dc70465e16a7ca830cd3898-sql'

In [ ]:
vn.train(question = "What are the top 10 most commonly mentioned skills?", sql="""
SELECT
  list.element AS skill, COUNT(*) AS frequency
FROM
  `hultaibigdata.jobs_data_analyst.postings`, UNNEST(description_tokens.list) AS list
GROUP BY skill
ORDER BY frequency DESC
LIMIT 10;
""")

'c3879d03f387289d6c2326ec65c0e712-sql'

In [ ]:
vn.train(question = "What are the top 5 most common job titles?", sql="""
SELECT
  title, COUNT(*)
FROM
  `hultaibigdata.jobs_data_analyst.postings`
GROUP BY title
ORDER BY COUNT(*) DESC
LIMIT 5;
""")

'19ab44e937db46a6e95ace8536721ba1-sql'

In [ ]:
vn.train(question = "What is the average annual salary for each of the top 15 job titles?", sql="""
SELECT title,
  AVG(salary_standardized) AS mean_salary
FROM `hultaibigdata.jobs_data_analyst.postings`
WHERE salary_standardized IS NOT NULL
GROUP BY title
ORDER BY mean_salary DESC
LIMIT 15;
""")

'c596bacabbbbfe286681228e026c4685-sql'

In [ ]:
vn.train(question = "What are the top 10 locations offering the highest salaries excluding working from home?", sql="""
SELECT location,
  AVG(salary_standardized) AS mean_salary
FROM `hultaibigdata.jobs_data_analyst.postings`
WHERE salary_standardized IS NOT NULL
AND (work_from_home IS NULL OR work_from_home IS FALSE)
GROUP BY location
ORDER BY mean_salary DESC
LIMIT 10;
""")

'59f09b2ed452833a7edeb75575592911-sql'

In [ ]:
# inspect training data
training_data = vn.get_training_data()
training_data

,id,training_data_type,question,content
0,319999-sql,sql,"Show titles, company names, and dates of 5 mos...","\nSELECT\n title, company_name, date_time\nFR..."
1,320000-sql,sql,Show the description and salary specification ...,"\nSELECT\n description, salary\nFROM\n `hult..."
2,320008-sql,sql,What are the top 10 most commonly mentioned sk...,"\nSELECT\n list.element AS skill, COUNT(*) AS..."
3,174282-ddl,ddl,None,\nCREATE TABLE `hultaibigdata.jobs_data_analys...
4,320007-sql,sql,What is the average salary of full-time jobs?,\nSELECT\n AVG(salary_standardized)\nFROM\n ...
5,320001-sql,sql,What is the mean annual salary among the posts...,\nSELECT\n AVG(salary_standardized)\nFROM\n ...
6,320006-sql,sql,What types of schedule are represented in the ...,\nSELECT\n DISTINCT schedule_type\nFROM\n `h...
7,319997-sql,sql,What percent of job posts provide salary infor...,\nSELECT\n 100*count(salary_standardized) / c...
8,320010-sql,sql,What is the average annual salary for each of ...,"\nSELECT title,\n AVG(salary_standardized) AS..."
9,319998-sql,sql,What percent of job posts appearing since Janu...,\nSELECT\n 100*count(salary_standardized) / c...


In [ ]:
vn.connect_to_bigquery(project_id='bigdata-ai2024pre-class')

In [11]:
from vanna.flask import VannaFlaskApp
VannaFlaskApp(vn, allow_llm_to_see_data=True).run()

Google Colab doesn't support running websocket servers. Disabling debug mode.


<IPython.core.display.Javascript object>

Your app is running at:
https://297fch08mj-496ff2e9c6d22116-8084-colab.googleusercontent.com/
 * Serving Flask app 'vanna.flask'
 * Debug mode: off
Info: None
None
SQL Prompt: [{'role': 'system', 'content': "You are a BigQuery SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\nCREATE TABLE `hultaibigdata.jobs_data_analyst.postings`\n(\n  title STRING,\n  company_name STRING,\n  location STRING,\n  via STRING,\n  description STRING,\n  extensions STRING,\n  posted_at STRING,\n  schedule_type STRING,\n  work_from_home BOOL,\n  salary STRING,\n  search_term STRING,\n  search_location STRING,\n  commute_time FLOAT64,\n  salary_pay STRING,\n  salary_rate STRING,\n  salary_avg FLOAT64,\n  salary_min FLOAT64,\n  salary_max FLOAT64,\n  salary_hourly FLOAT64,\n  salary_yearly FLOAT64,\n  salary_standardized FLOAT64,\n  description_tokens STRUCT